In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import tensorflow as tf
import keras
import pysmiles
import json
import networkx as nx
import random

In [ ]:
import time

In [ ]:
import logging
logging.getLogger('pysmiles').setLevel(logging.CRITICAL)

Load and pre-process data

In [ ]:
de_data_train = pq.read_table("../data/de_train.parquet").to_pandas()
de_data_train

In [ ]:
de_data_train["cell_type"].unique()[1]

In [ ]:
cellNameToInt = {de_data_train["cell_type"].unique()[i]: i for i in range(len(de_data_train["cell_type"].unique()))}

In [ ]:
de_data_train["cell_type_int"] = de_data_train["cell_type"].map(cellNameToInt)

In [ ]:
cellNameToInt

In [ ]:
gene_names = de_data_train.columns[5:-2]

## Divide into train and test

In [ ]:
# Cell types where all (cell_type, sm) pairs will be used for training
train_only_cell_types     = ["T cells CD4+", "T cells CD8+", "T regulatory cells"]
# Cell types where only some (cell_type, sm) pairs will be used for training
train_and_test_cell_types = ["B cells", "Myeloid cells", "NK cells"]

In [ ]:
# Create a dict mapping cell_name -> list of sm given for cell_name
sm_names_by_cell_type = de_data_train.groupby("cell_type")["sm_name"].unique().to_dict()
# Get list of small molecules given for cell types with a reduced set of (cell_type, sm) pairs
train_and_test_sm = sm_names_by_cell_type["B cells"]

In [ ]:
# For cell types where only some (cell_type, sm) pairs will be used for training
# Choose which small molecules will be used for training and which for test
num_b_sm       = len(sm_names_by_cell_type["B cells"])
num_myeloid_sm = len(sm_names_by_cell_type["Myeloid cells"])
num_nk_sm      = len(sm_names_by_cell_type["NK cells"])

b_cell_train       = sm_names_by_cell_type["B cells"][:num_b_sm//2]
myeloid_cell_train = sm_names_by_cell_type["Myeloid cells"][:num_myeloid_sm//2]
nk_cell_train      = sm_names_by_cell_type["NK cells"][:num_nk_sm//2]

b_cell_test       = sm_names_by_cell_type["B cells"][num_b_sm//2:]
myeloid_cell_test = sm_names_by_cell_type["Myeloid cells"][num_myeloid_sm//2:]
nk_cell_test      = sm_names_by_cell_type["NK cells"][num_nk_sm//2:]

In [ ]:
# Create training combinations with all (cell_type, sm) pairs for train only cell types
training_combinations = dict((cell_type, sm_names_by_cell_type[cell_type]) for cell_type in train_only_cell_types)

In [ ]:
# Include training (cell_type, sm) pairs from train_test cell types
training_combinations["B cells"] = b_cell_train
training_combinations["Myeloid cells"] = myeloid_cell_train
training_combinations["NK cells"] = nk_cell_train

In [ ]:
# Create testing combinations
testing_combinations = {}
testing_combinations["B cells"] = b_cell_test
testing_combinations["Myeloid cells"] = myeloid_cell_test
testing_combinations["NK cells"] = nk_cell_test

In [ ]:
training_combinations

In [ ]:
testing_combinations

In [ ]:
# Convert into (cell_type, sm) pairs
training_pairs = set({})
for cell_type in training_combinations.keys():
    for sm in training_combinations[cell_type]:
            for gene_name in gene_names:
                training_pairs.add((cell_type+", "+sm, gene_name))

testing_pairs = set({})
for cell_type in testing_combinations.keys():
    for sm in testing_combinations[cell_type]:
            for gene_name in gene_names:
                testing_pairs.add((cell_type+", "+sm, gene_name))

list(training_pairs)[:10]

In [ ]:
de_data_train["cell_type_sm_pair"] = de_data_train["cell_type"]+", "+de_data_train["sm_name"]

In [ ]:
de_data_train[de_data_train["cell_type_sm_pair"] == "T regulatory cells, FK 866"].iloc[0][5:-2]

In [ ]:
sequences_file = "../data/sequences_int.jsonl"

gene_symbol_to_id = {}
gene_sequences = []

with open(sequences_file, "r") as sequences:
    i = 0
    for line in sequences:
        json_line = json.loads(line)
        gene_sequences.append(json_line["seq"])
        gene_symbol_to_id[json_line["gene"]] = i
        i += 1

In [ ]:
gene_sequences = tf.convert_to_tensor(gene_sequences)

In [ ]:
all_molecules = de_data_train["sm_name"].unique()

In [ ]:
((32*100000)/8)/1024

# Create Dataset Generator

In [ ]:
import GraphLayers

In [ ]:
MAX_NODES = 150
MAX_EDGES = 200
EMBEDDING_DIM = 120

MAX_DNA_LEN = 40000

In [ ]:
def smiles_to_graph(smiles_molecule):
    graph = pysmiles.read_smiles(smiles_molecule, explicit_hydrogen=True)
    return GraphLayers.convertFromNetworkX(graph, 
                               MAX_NODES,
                               MAX_EDGES, 
                               EMBEDDING_DIM)

In [ ]:
mol_to_id = {}
mol_vertices = []
mol_edges = []
mol_unis = []
mol_adjs = []
mol_conns = []
mol_edge_adjs = []

i = 0
for mol in all_molecules:
    smiles = de_data_train[de_data_train["sm_name"] == mol].iloc[0]["SMILES"]
    mol_to_id[mol] = i
    mol_ver, mol_edj, mol_uni, mol_am, mol_conn, mol_edgeAdj = smiles_to_graph(smiles)

    mol_vertices.append(mol_ver)
    mol_edges.append(mol_edj)
    mol_unis.append(mol_uni)
    mol_adjs.append(mol_am)
    mol_conns.append(mol_conn)
    mol_edge_adjs.append(mol_edgeAdj)
    i += 1

In [ ]:
mol_vertices = tf.convert_to_tensor(mol_vertices)
mol_edges = tf.convert_to_tensor(mol_edges)
mol_unis = tf.convert_to_tensor(mol_unis)
mol_adjs = tf.convert_to_tensor(mol_adjs)
mol_conns = tf.convert_to_tensor(mol_conns)
mol_edge_adjs = tf.convert_to_tensor(mol_edge_adjs)

In [ ]:
mol_gene_cell_to_de = {}
for index, example in de_data_train.iterrows():
    mol_id = mol_to_id[example["sm_name"]]
    cell_type = example["cell_type_int"]
    for gene_name in gene_names:
        gene_id = gene_symbol_to_id[gene_name]
        mol_gene_cell_to_de[(mol_id, gene_id, cell_type)] = example[gene_name]

In [ ]:
for cell_type in testing_combinations.keys():
    print(cell_type)

In [ ]:
training_pairs = set({})

for cell_type in training_combinations.keys():
    for mol_name in training_combinations[cell_type]:
            for gene_name in gene_names:
                mol_id = mol_to_id[mol_name]
                gene_id = gene_symbol_to_id[gene_name]
                cell_id = cellNameToInt[cell_type]
                
                training_pairs.add((mol_id, gene_id, cell_id))

testing_pairs = set({})

for cell_type in testing_combinations.keys():
    for mol_name in testing_combinations[cell_type]:
            for gene_name in gene_names:
                mol_id = mol_to_id[mol_name]
                gene_id = gene_symbol_to_id[gene_name]
                cell_id = cellNameToInt[cell_type]
                
                testing_pairs.add((mol_id, gene_id, cell_id))

In [ ]:
from numpy.random import permutation

In [ ]:
training_mol_ids = []
training_gene_ids = []
training_cell_types = []
training_de_vals = []

training_pairs = list(training_pairs)

for i in permutation(len(training_pairs)):
    mol_id, gene_id, cell_id = training_pairs[i]
    de = mol_gene_cell_to_de[(mol_id, gene_id, cell_id)]
    
    training_mol_ids.append(mol_id)
    training_gene_ids.append(gene_id)
    training_cell_types.append(cell_id)
    training_de_vals.append(de)

testing_mol_ids = []
testing_gene_ids = []
testing_cell_types = []
testing_de_vals = []

testing_pairs = list(testing_pairs)

for i in permutation(len(testing_pairs)):
    mol_id, gene_id, cell_id = testing_pairs[i]
    de = mol_gene_cell_to_de[(mol_id, gene_id, cell_id)]
    
    testing_mol_ids.append(mol_id)
    testing_gene_ids.append(gene_id)
    testing_cell_types.append(cell_id)
    testing_de_vals.append(de)

In [ ]:
training_mol_ids = tf.convert_to_tensor(training_mol_ids)
training_gene_ids = tf.convert_to_tensor(training_gene_ids)
training_cell_types = tf.convert_to_tensor(training_cell_types)
training_de_vals = tf.convert_to_tensor(training_de_vals)

testing_mol_ids = tf.convert_to_tensor(testing_mol_ids)
testing_gene_ids = tf.convert_to_tensor(testing_gene_ids)
testing_cell_types = tf.convert_to_tensor(testing_cell_types)
testing_de_vals = tf.convert_to_tensor(testing_de_vals)

In [ ]:
def get_gene_sequences(in_mol_ids, in_gene_ids, in_cell_types, in_de_vals):
    current_gene_sequences = tf.gather(gene_sequences, in_gene_ids)
    return in_mol_ids, in_gene_ids, current_gene_sequences, in_cell_types, in_de_vals

In [ ]:
def get_mol_graphs(in_mol_ids, in_gene_ids, in_gene_seqs, in_cell_types, in_de_vals):
    current_mol_vertices = tf.gather(mol_vertices, in_mol_ids)
    current_mol_edges = tf.gather(mol_edges, in_mol_ids)
    current_mol_unis = tf.gather(mol_unis, in_mol_ids)
    current_mol_adjs = tf.gather(mol_adjs, in_mol_ids)
    current_mol_conns = tf.gather(mol_conns, in_mol_ids)
    current_mol_edge_adjs = tf.gather(mol_edge_adjs, in_mol_ids)
    
    return current_mol_vertices, current_mol_edges, current_mol_unis, current_mol_adjs, current_mol_conns, current_mol_edge_adjs, in_gene_ids, in_gene_seqs, in_cell_types, in_de_vals

In [ ]:
def name_tensors(in_mol_vertices, 
                 in_mol_edges, 
                 in_mol_unis, 
                 in_mol_adjs, 
                 in_mol_conns, 
                 in_mol_edge_adjs,
                 in_gene_ids,
                 in_gene_seqs, 
                 in_cell_types, 
                 in_de_vals):
    return {
        "mol_ver": in_mol_vertices,
        "mol_edj": in_mol_edges,
        "mol_uni": in_mol_unis,
        "mol_am": in_mol_adjs,
        "mol_conn": in_mol_conns,
        "mol_edgeAdj": in_mol_edge_adjs,
        "gene_id": in_gene_ids,
        "dna_seq": in_gene_seqs,
        "cell_type": in_cell_types
    }, in_de_vals

In [ ]:
from GraphLayers import *

In [ ]:
def build_model(params):
    vertices = Input(shape=((MAX_NODES, EMBEDDING_DIM,)), name="mol_ver")
    edges = Input(shape=((MAX_EDGES, EMBEDDING_DIM,)), name="mol_edj")
    universal = Input(shape=((EMBEDDING_DIM,)), name="mol_uni")
    adj = Input(shape=((MAX_NODES, MAX_NODES,)), name="mol_am")
    conEd = Input(shape=((MAX_NODES, MAX_EDGES,)), name="mol_conn")
    edgeAdj = Input(shape=((MAX_EDGES, MAX_EDGES,)), name="mol_edgeAdj")
    dna_sequence = Input(shape=((40000,)), name="dna_seq")
    geneID = Input(shape=((1,)), name="gene_id")
    cellType = Input(shape=((1,)), name="cell_type")
    
    x = [vertices, edges, universal, adj, conEd, edgeAdj]

    for i in range(params["graph_layers"]):
        for k in range(params["pool_steps"]):
            x = PoolStep(params[f"step_{k}_pve"],
                        params[f"step_{k}_pee"],
                        params[f"step_{k}_pue"],
                        params[f"step_{k}_pvv"],
                        params[f"step_{k}_pev"],
                        params[f"step_{k}_puv"],
                        params[f"step_{k}_pvu"],
                        params[f"step_{k}_peu"])(x)
        x = GraphUpdate(params["embedding_dim"], params["embedding_dim"], params["embedding_dim"], params["update_function_depth"], activation="relu", dropout=params["dropout"])(x)

    x = PoolStep(p_ve=False,
                p_ee=False,
                p_ue=False,
                p_vv=False,
                p_ev=False,
                p_uv=False,
                p_vu=True,
                p_eu=True)(x)
    
    u = x[2]

    final_tensors = [u, cellType]
    
    if params["use_gene_sequence"]:
        dna_seq = Dense(64)(dna_sequence)
        dna_seq = tf.keras.layers.LeakyReLU()(dna_seq)
        dna_seq = tf.keras.layers.BatchNormalization()(dna_seq)
        
        dna_seq = Dense(32)(dna_seq)
        dna_seq = tf.keras.layers.LeakyReLU()(dna_seq)
        dna_seq = tf.keras.layers.BatchNormalization()(dna_seq)
        
        dna_seq = Dense(16)(dna_seq)
        dna_seq = tf.keras.layers.LeakyReLU()(dna_seq)
        dna_seq = tf.keras.layers.BatchNormalization()(dna_seq)
        
        final_tensors.append(dna_seq)
        
    if params["use_gene_id"]:
        final_tensors.append(geneID)

    u = Concatenate()(final_tensors)
    
    for i in range(params["num_final_layers"]):
        u = Dense(16)(u)
        u = tf.keras.layers.LeakyReLU()(u)
        u = tf.keras.layers.BatchNormalization()(u)

    u = Dense(1)(u)
    
    return Model(inputs=[vertices, edges, universal, adj, conEd, edgeAdj, geneID, cellType, dna_sequence], outputs=u)

In [ ]:
def generate_parameter_set():
    params = {}

    params["graph_layers"] = random.randint(1, 5)
    params["pool_steps"] = random.randint(3, 5)

    params["update_function_depth"] = random.randint(1, 5)

    for k in range(params["pool_steps"]):
        params[f"step_{k}_pve"] = random.choice([True, False])
        params[f"step_{k}_pee"] = random.choice([True, False])
        params[f"step_{k}_pue"] = random.choice([True, False])
        params[f"step_{k}_pvv"] = random.choice([True, False])
        params[f"step_{k}_pev"] = random.choice([True, False])
        params[f"step_{k}_puv"] = random.choice([True, False])
        params[f"step_{k}_pvu"] = random.choice([True, False])
        params[f"step_{k}_peu"] = random.choice([True, False])

    #params["embedding_dim"] = random.choice([10, 50, 60, 70, 80, 90, 100])
    params["embedding_dim"] = 64
    params["num_final_layers"] = random.randint(1, 5)

    params["optimizer"] = random.choice(["Adam", "SGD"])
    params["optimizer"] = "Adam"
    
    if params["optimizer"] == "RMSProp":
        #params["learning_rate"] = random.uniform(0.0001, 0.1)
        params["learning_rate"] = 0.001

    if params["optimizer"] == "Adam":
        #params["learning_rate"] = random.uniform(0.00001, 0.1)
        params["learning_rate"] = random.choice([0.0001, 0.00001])

    if params["optimizer"] == "SGD":
        #params["learning_rate"] = random.uniform(0.001, 0.1)
        params["learning_rate"] = random.choice([0.01, 0.001])

    params["batch_size"] = 512
    params["dropout"] = random.choice([True, False])
    params["dropout"] = True
    
    #params["use_gene_sequence"] = random.choice([True, True, False])
    #params["use_gene_id"] = random.choice([True, True, False])
    params["use_gene_sequence"] = random.choice([True, False])
    params["use_gene_id"] = True
    
    return params

In [ ]:
params = generate_parameter_set()

model = build_model(params)

if params["optimizer"] == "RMSProp":
    optimizer=tf.keras.optimizers.RMSprop(params["learning_rate"], clipnorm=1)

if params["optimizer"] == "Adam":
    optimizer=tf.keras.optimizers.Adam(params["learning_rate"], clipnorm=1)

if params["optimizer"] == "SGD":
    optimizer=tf.keras.optimizers.SGD(params["learning_rate"], clipnorm=1)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.MeanSquaredError(),
)

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices((training_mol_ids, 
                                                       training_gene_ids, 
                                                       training_cell_types, 
                                                       training_de_vals))
training_dataset = training_dataset.batch(512)
training_dataset = training_dataset.map(get_gene_sequences, num_parallel_calls=tf.data.AUTOTUNE)
training_dataset = training_dataset.map(get_mol_graphs, num_parallel_calls=tf.data.AUTOTUNE)
training_dataset = training_dataset.map(name_tensors, num_parallel_calls=tf.data.AUTOTUNE)
training_dataset = training_dataset.prefetch(tf.data.AUTOTUNE)

testing_dataset = tf.data.Dataset.from_tensor_slices((testing_mol_ids, 
                                                       testing_gene_ids, 
                                                       testing_cell_types, 
                                                       testing_de_vals))
testing_dataset = testing_dataset.batch(512)
testing_dataset = testing_dataset.map(get_gene_sequences, num_parallel_calls=tf.data.AUTOTUNE)
testing_dataset = testing_dataset.map(get_mol_graphs, num_parallel_calls=tf.data.AUTOTUNE)
testing_dataset = testing_dataset.map(name_tensors, num_parallel_calls=tf.data.AUTOTUNE)
testing_dataset = testing_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
from keras.utils.layer_utils import count_params

In [ ]:
stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
nan_stopper = tf.keras.callbacks.TerminateOnNaN()

start = time.time()
history = model.fit(training_dataset, validation_data=testing_dataset, epochs=100, callbacks=[stopper, nan_stopper])
train_time = time.time()-start

if "val_loss" not in history.history:
    log_line = {"time": time.time(),
            "config": params, 
            "history": history.history,
            "nan_fail": True}
    with open("logfile_compute.jsonl", "a") as logfile:
        logfile.write(json.dumps(log_line)+"\n")

else:
    lowest_loss = min(history.history["val_loss"])
    train_steps = history.history["val_loss"].index(lowest_loss)
    
    trainable_params = sum(count_params(layer) for layer in model.trainable_weights)
    non_trainable_params = sum(count_params(layer) for layer in model.non_trainable_weights)
    
    log_line = {"time": time.time(),
                "config": params, 
                "history": history.history, 
                "lowest_val_loss": lowest_loss, 
                "train_steps": train_steps, 
                "train_time": train_time,
                "trainable_params": trainable_params,
                "non_trainable_params": non_trainable_params}
    
    with open("logfile_compute.jsonl", "a") as logfile:
        logfile.write(json.dumps(log_line)+"\n")